In [1]:
from pyecharts import options as opts
from pyecharts.charts import PictorialBar, Line, Bar
from pyecharts.globals import SymbolType
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
from pyecharts.commons.utils import JsCode
import pandas as pd
import re

In [2]:
provincial = pd.read_csv('provincial_capital.csv')  # 读取省会信息
china_city_code = pd.read_csv('china-city-list.csv')  # 读取城市 id 信息
weather = pd.read_csv("weather_data.csv")  # 读取天气信息

In [3]:
# 查看天气总体情况
weather['wea'].value_counts()

# 转换天气变量，数值越多，说明降水概率越大
weather_dict = {
    "snow": 100,
    "rain": 80,
    "cloud": 50,
    "overcast": 60,
    "sun": 20
}


In [4]:
# 提取省会城市 id
provincial_data = pd.DataFrame()
for i in provincial['city'].values.tolist():
    for j in china_city_code['City_CN'].values.tolist():
        if j == i:
            provincial_data = pd.concat([china_city_code[china_city_code['City_CN'] == j], provincial_data])
provincial_data

,City_ID,City_EN,City_CN,Country_code,Country_EN,Country_CN,Province_EN,Province_CN,Admin_ district_EN,Admin_ district_CN,Latitude,Longitude,AD_code,Unnamed: 13
1394,CN101160101,lanzhou,兰州,CN,China,中国,gansu,甘肃,lanzhou,兰州,36.058040,103.823555,"620101,620100,620000",NaN
3188,CN101310101,haikou,海口,CN,China,中国,hainan,海南,haikou,海口,20.031971,110.331190,"460101,460100,460000",NaN
3225,CN101340101,Taibei,台北,CN,China,中国,taiwan,台湾,taibei,台北,25.040000,121.516000,710000,NaN
3222,CN101330101,macao,澳门,CN,China,中国,macao,澳门,macao,澳门,22.202000,113.544000,"820000,820001,820002,820003,820004,820005,820007",NaN
3219,CN101320101,hongkong,香港,CN,China,中国,hongkong,香港,hongkong,香港,22.307000,114.177000,"810000,810001,810002,810003,810004,810005,8100...",NaN
2138,CN101230101,fuzhou,福州,CN,China,中国,fujian,福建,fuzhou,福州,26.075302,119.306240,"350101,350100,350000",NaN
1915,CN101210101,hangzhou,杭州,CN,China,中国,zhejiang,浙江,hangzhou,杭州,30.287458,120.153580,"330101,330100,330000",NaN
1491,CN101170101,yinchuan,银川,CN,China,中国,ningxia,宁夏,yinchuan,银川,38.466370,106.278175,"640101,640100,640000",NaN
736,CN101100101,taiyuan,太原,CN,China,中国,shanxi,山西,taiyuan,太原,37.857014,112.549250,"140101,140100,140000",NaN
980,CN101120101,jinan,济南,CN,China,中国,shandong,山东,jinan,济南,36.675808,117.000920,"370101,370100,370000",NaN


In [5]:
# 按照城市分组
wea_group = weather.groupby('city').apply(lambda x: x[:])
wea_group

city     time   wea      tem    wind wind_level
city                                                    
上海   248   上海  周四（12日）  多云转晴  30℃/23℃  北风转西北风        <3级
     249   上海  周五（13日）  雨转多云  31℃/24℃  东风转东北风   3-4级转<3级
     250   上海  周六（14日）  雨转多云  31℃/25℃  东风转东北风   3-4级转<3级
     251   上海  周日（15日）     雨  30℃/25℃     东北风       3-4级
     252   上海  周一（16日）     雨  29℃/25℃     东北风       3-4级
...       ...      ...   ...      ...     ...        ...
香港   35    香港  周日（15日）     雨  30℃/28℃  西北风转南风   <3级转4-5级
     36    香港  周一（16日）     雨  28℃/26℃      东风   3-4级转<3级
     37    香港  周二（17日）     雨  29℃/27℃  东北风转东风        <3级
     38    香港  周三（18日）     雨  30℃/27℃      东风   <3级转3-4级
     39    香港  周四（19日）     雨  30℃/27℃   北风转西风  3-4级转4-5级

[272 rows x 6 columns]

In [6]:
# 提取中秋节当天天气信息
zhongqiu = wea_group[wea_group['time'] == '周五（13日）']
zhongqiu

,,city,time,wea,tem,wind,wind_level
city,,,,,,,
上海,249,上海,周五（13日）,雨转多云,31℃/24℃,东风转东北风,3-4级转<3级
乌鲁木齐,177,乌鲁木齐,周五（13日）,阴转多云,20℃/9℃,北风转南风,<3级
兰州,1,兰州,周五（13日）,雨,18℃/12℃,东北风,<3级
北京,265,北京,周五（13日）,雨,29℃/19℃,西南风转北风,<3级
南京,81,南京,周五（13日）,多云,31℃/23℃,东北风转东风,<3级
南宁,129,南宁,周五（13日）,雨,31℃/23℃,北风转西北风,<3级
南昌,97,南昌,周五（13日）,晴,35℃/25℃,东北风转北风,<3级
台北,17,台北,周五（13日）,雨,29℃/26℃,东北风转东风,4-5级
合肥,89,合肥,周五（13日）,多云转晴,31℃/22℃,东风,<3级


In [7]:
# 城市信息
zhongqiu_city = zhongqiu['city'].values.tolist()
zhongqiu_city

['上海',
 '乌鲁木齐',
 '兰州',
 '北京',
 '南京',
 '南宁',
 '南昌',
 '台北',
 '合肥',
 '呼和浩特',
 '哈尔滨',
 '天津',
 '太原',
 '广州',
 '成都',
 '拉萨',
 '昆明',
 '杭州',
 '武汉',
 '沈阳',
 '济南',
 '海口',
 '澳门',
 '石家庄',
 '福州',
 '西宁',
 '西安',
 '贵阳',
 '郑州',
 '重庆',
 '银川',
 '长春',
 '长沙',
 '香港']

In [8]:
# 获取温度信息
rege = r'(\d+)℃/(\d+)℃'


def trans_tem(tem):
    tmp_tem = re.match(rege, tem)
    mid_tem = (int(tmp_tem.group(1)) + int(tmp_tem.group(2)))/2
    return mid_tem


# 转换降水信息
def check_weather(wea):
    if wea[-1:] == '晴':
        wea = weather_dict['sun']
    elif wea[-1:] == '云':
        wea = weather_dict['cloud']
    elif wea[-1:] == '雨':
        wea = weather_dict['rain']
    elif wea[-1:] == '阴':
        wea = weather_dict['overcast']
    return wea

In [9]:
# 获取降水和温度信息
weather_data = map(check_weather, zhongqiu['wea'].values.tolist())
weather_data = list(weather_data)
tem_data = map(trans_tem, zhongqiu['tem'].values.tolist())
tem_data = list(tem_data)
weather_data_list = list(weather_data)

In [10]:
# 几大城市天气情况
weather = pd.read_csv("weather_data.csv")
beijing = weather[weather['city'] == '北京']
shanghai = weather[weather['city'] == '上海']
hangzhou = weather[weather['city'] == '杭州']
chengdu = weather[weather['city'] == '成都']

time_shanghai = shanghai['time'].values.tolist()
wea_shanghai = shanghai['wea'].values.tolist()
wea_list_shanghai = list(map(check_weather, wea_shanghai))
tem_shanghai = shanghai['tem'].values.tolist()
tem_list_shanghai = list(map(trans_tem, tem_shanghai))
line = Line()
line.add_xaxis(time_shanghai)
line.add_yaxis("降水概率", wea_list_shanghai, is_smooth=True,
              markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]))
line.add_yaxis("温度", tem_list_shanghai,
              markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]))
line.set_global_opts(
            title_opts=opts.TitleOpts(title="上海中秋节前后降水与温度"),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30), is_show=True)
)
line.render_notebook()